In [ ]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.tree import _tree
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import LeaveOneGroupOut, cross_validate
from sklearn.metrics import accuracy_score, f1_score, make_scorer, confusion_matrix
from scipy.stats import wilcoxon
from matplotlib import pyplot as plt

---
__UTILITY FUNCTIONS__

---

In [ ]:
#converts tree in a readable list of rules 
def tree_to_code(tree, feature_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]
    print("def tree({}):".format(", ".join(feature_names)))

    def recurse(node, depth):
        indent = "  " * depth
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            print("{}if {} <= {}:".format(indent, name, threshold))
            recurse(tree_.children_left[node], depth + 1)
            print("{}elif {} > {}".format(indent, name, threshold))
            recurse(tree_.children_right[node], depth + 1)
        else:
            print("{}return {}".format(indent, np.argmax(tree_.value[node])))

    recurse(0, 1)

In [ ]:
def scatter_plot(df, var1, var2, tester):
    x = df[var1]
    y = df[var2]
    classes = df['Class']

    map = {0.0: 'Left', 1.0: 'Centered', 2.0: 'Right'}

    # Replace values in the array
    string_array = np.where(np.isin(classes, list(map.keys())), [map[val] for val in classes], classes)

    unique = list(set(string_array))
    
    # Assign colors based on unique values
    colors = plt.cm.get_cmap('jet', len(unique))

    for i, u in enumerate(unique):
        xi = [x[j] for j in range(len(x)) if string_array[j] == u]
        yi = [y[j] for j in range(len(x)) if string_array[j] == u]
        plt.scatter(xi, yi, c=[colors(i)], label=str(u))

    plt.xlabel(var1)  # Fix: 'set_xlabel' should be 'xlabel'
    plt.ylabel(var2)  # Fix: 'set_ylabel' should be 'ylabel'

    plt.title(str(tester))

    plt.legend()
    plt.show()

---
__DATA LOADING__

---

In [ ]:
Lorenzo = pd.read_csv("./../Dati_unificati/lorenzo_smooth.csv")
Leo = pd.read_csv("./../Dati_unificati/leo_smooth.csv")
Irene = pd.read_csv("./../Dati_unificati/irene_smooth.csv")
Carlotta = pd.read_csv("./../Dati_unificati/carlotta_smooth.csv")

ds = pd.concat([Lorenzo, Leo, Irene, Carlotta], ignore_index=True)

X = ds.drop(columns=['Class', 'Tester'])
y = ds['Class']
groups = ds['Tester']
print(X.shape)
print(y.shape)
print(groups.shape)

print(groups.drop_duplicates())

loso_cv = LeaveOneGroupOut()

---
__KNN__

---

In [ ]:
results_KNN = []

#gives 800 values of test_fscore to be checked with Wilcoxon signed rank test
for i in range(1, 201):    
    classifier_KNN = KNeighborsClassifier(n_neighbors=6)
    scaler = RobustScaler()
    feat_sel = SelectKBest(k=2)

    estimators_KNN = [('scaling', scaler), ('feature-selection', feat_sel), ('clf', classifier_KNN)]
    pipe_KNN = Pipeline(estimators_KNN)

    scores_KNN = cross_validate(pipe_KNN,
                                X,
                                y,
                                return_estimator=True,
                                cv=loso_cv,
                                n_jobs=-1,
                                groups=groups,
                                error_score="raise",
                                scoring={'fscore': make_scorer(f1_score, average='weighted'),
                                            'accuracy': make_scorer(accuracy_score)}
                                )
    results_KNN.append(pd.DataFrame(scores_KNN))

# Concatenate results outside the loop
final_results_KNN = pd.concat(results_KNN, ignore_index=True)

print(final_results_KNN)

---
__RANDOM FOREST__

---

In [ ]:
results_RF = []  # List to store results

#gives 800 values of test_fscore to be checked with Wilcoxon signed rank test
for i in range(1, 201):
    classifier_RF = RandomForestClassifier(n_estimators=61, min_samples_split=30, min_samples_leaf=1, max_features='sqrt', bootstrap=True, max_depth=10, random_state=i)
    scaler = RobustScaler()
    feat_sel = SelectKBest(k=2)
    estimators_RF = [('scaling', scaler), ('feature-selection', feat_sel), ('clf', classifier_RF)]
    pipe_RF = Pipeline(estimators_RF)

    scores_RF = cross_validate(pipe_RF,
                               X,
                               y,
                               return_estimator=True,
                               cv=loso_cv,
                               n_jobs=-1,
                               groups=groups,
                               error_score="raise",
                               scoring={'fscore': make_scorer(f1_score, average='weighted'),
                                        'accuracy': make_scorer(accuracy_score)}
                               )
    
    results_RF.append(pd.DataFrame(scores_RF))

# Concatenate results outside the loop
final_results_RF = pd.concat(results_RF, ignore_index=True)

print(final_results_RF)
print(final_results_RF.iloc[final_results_RF['test_fscore'].idxmin()])

---
__ADABOOST__

---

In [ ]:
results_AB = []

#gives 800 values of test_fscore to be checked with Wilcoxon signed rank test
for i in range(1,201):
    classifier_AdaBoost = AdaBoostClassifier(estimator=RandomForestClassifier(n_estimators=61, min_samples_split=30, min_samples_leaf=1, max_features='sqrt', bootstrap=True, max_depth=10, random_state=i), n_estimators=83, random_state=i)
    scaler = RobustScaler()
    feat_sel = SelectKBest(k=2)

    estimators_AdaBoost = [('scaling', scaler), ('feature-selection', feat_sel), ('clf', classifier_AdaBoost)]
    pipe_AdaBoost = Pipeline(estimators_AdaBoost)

    scores_AdaBoost = cross_validate(pipe_AdaBoost,
                            X,
                            y,
                            return_estimator = True,
                            cv = loso_cv,
                            n_jobs=-1,
                            groups=groups,
                            error_score="raise",
                            scoring = {'fscore': make_scorer(f1_score, average='weighted'),
                                        'accuracy': make_scorer(accuracy_score)}
    )
    results_AB.append(pd.DataFrame(scores_AdaBoost))
    
# Concatenate results outside the loop
final_results_AdaBoost = pd.concat(results_AB, ignore_index=True)
print(final_results_AdaBoost)

---
__PERFORMANCE EVALUATION__

---

In [ ]:
metrics = pd.DataFrame({'KNN': final_results_KNN['test_fscore'],
                        'RF': final_results_RF['test_fscore'],
                        'AB': final_results_AdaBoost['test_fscore']})
metrics

In [ ]:
#boxplot of the results of the three models
ax = metrics.boxplot(figsize = (3,3))
ax.set_ylabel('f-score')
plt.show()

In [ ]:
print(wilcoxon(metrics.KNN, metrics.RF))

In [ ]:
print(wilcoxon(metrics.KNN, metrics.AB))

In [ ]:
print(wilcoxon(metrics.RF, metrics.AB))

In [ ]:
print("Mean of RF: " + str(np.mean(metrics.RF)))
print("Mean of AB: " + str(np.mean(metrics.AB)))

---
__TREE ANALYSIS OF ADABOOST__

---

In [ ]:
#creates a classifier that tries to predict the tester with the worst performances to see why

classifier_AdaBoost = AdaBoostClassifier(estimator=RandomForestClassifier(n_estimators=61, min_samples_split=30, min_samples_leaf=1, max_features='sqrt', bootstrap=True, max_depth=10), n_estimators=83, random_state=0)
scaler = RobustScaler()
feat_sel = SelectKBest(k=2)

estimators_AdaBoost = [('scaling', scaler), ('feature-selection', feat_sel), ('clf', classifier_AdaBoost)]
pipe_AdaBoost_tree = Pipeline(estimators_AdaBoost)

X_train = ds[(ds['Tester'] != 'Lorenzo')].drop(columns=['Class', 'Tester'])
X_test= ds[(ds['Tester'] == 'Lorenzo')].drop(columns=['Class', 'Tester'])

y_train = ds[(ds['Tester'] != 'Lorenzo')]['Class']
y_test= ds[(ds['Tester'] == 'Lorenzo')]['Class']

pipe_AdaBoost_tree.fit(X_train, y_train)
y_pred = pipe_AdaBoost_tree.predict(X_test)

print(accuracy_score(y_pred, y_test))

#the selected k=2 best feature chosen by the model
print(X.columns[pipe_AdaBoost_tree['feature-selection'].get_support()])

In [ ]:
#visulization of the tree rules
tree_to_code(pipe_AdaBoost_tree['clf'].estimators_[0].estimators_[31], X.columns[pipe_AdaBoost_tree['feature-selection'].get_support()])

In [ ]:
scaler.fit_transform(X_test[[X.columns[pipe_AdaBoost_tree['feature-selection'].get_support()][0], X.columns[pipe_AdaBoost_tree['feature-selection'].get_support()][1]]])

In [ ]:
feature1 = X.columns[pipe_AdaBoost_tree['feature-selection'].get_support()][0]
feature2 = X.columns[pipe_AdaBoost_tree['feature-selection'].get_support()][1]
print(feature1)
print(feature2)

In [ ]:
feature1 = X.columns[pipe_AdaBoost_tree['feature-selection'].get_support()][0]
feature2 = X.columns[pipe_AdaBoost_tree['feature-selection'].get_support()][1]
tree_test = X_test[[feature1, feature2]]
tree_test_scaled = pd.DataFrame(scaler.fit_transform(tree_test))
print(tree_test_scaled)

In [ ]:
y_pred_tree = pipe_AdaBoost_tree['clf'].estimators_[0].estimators_[31].predict(tree_test_scaled)
cm = confusion_matrix(y_test, y_pred_tree)

In [ ]:
#scale the lorenzo dataset to confront it agaist the tree rules
columns = ds[(ds['Tester'] == 'Lorenzo')].drop(columns=['Class', 'Tester']).columns
class_col = ds[(ds['Tester'] == 'Lorenzo')]['Class']

scaled_lorenzo = pd.DataFrame(scaler.fit_transform(ds[(ds['Tester'] == 'Lorenzo')].drop(columns=['Class', 'Tester'])))
scaled_lorenzo.columns = columns
scaled_lorenzo['Class'] = class_col

In [ ]:
var1 = X.columns[pipe_AdaBoost['feature-selection'].get_support()][0]
var2 = X.columns[pipe_AdaBoost['feature-selection'].get_support()][1]

scatter_plot(scaled_lorenzo, var1, var2, 'Lorenzo')